In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.metrics import fbeta_score, make_scorer
from scipy.stats import chi2_contingency,kruskal,kendalltau
from sklearn.metrics import precision_recall_curve, average_precision_score
from matplotlib.colors import LinearSegmentedColormap

### **Split the data into training and test sets**

In [ ]:
imputed_result = pd.read_csv('/Users/xxx/Desktop/LISS data/Cleaned_data.csv')

train_data, test_data = train_test_split(
    imputed_result,
    test_size=0.2,
    random_state=42,
    stratify=imputed_result[['hypertension', 'age_group', 'male']]
)

test_male = test_data[test_data['male'] == 1]
test_female = test_data[test_data['male'] == 0]
test_age_1 = test_data[test_data['age_group'] == 1]
test_age_2 = test_data[test_data['age_group'] == 2]
test_age_3 = test_data[test_data['age_group'] == 3]
test_age_4 = test_data[test_data['age_group'] == 4]
test_age_5 = test_data[test_data['age_group'] == 5]

In [ ]:
# Distrubution of age and sex 
print(test_data['age_group'].value_counts(dropna=False))
print(test_data['male'].value_counts(normalize=True) * 100)
print(test_data['male'].value_counts(dropna=False))
print(test_data['male'].value_counts(normalize=True) * 100)

### **Statistical analysis**

In [ ]:
# Define the list of variables to test
variables_to_test = [
    'age_group', 'male', 'chest_pain', 'angina', 'heart_dis', 
    'stroke', 'h_cholesterol', 'diabetes', 'smk', 'drink', 
    'marriage_x', 'pri_occuption_x', 'net_income_cls3', 
    'debt_total_cls2', 'asset_total_cls', 'depression_y', 
    'depression', 'anxiety', 'edu_cls', 'sleeping_prb', 
    'happiness', 'calmness', 'life_statistician_x','asset_total_cls_c'
]


# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

# Perform Chi-square test for each variable
for var in variables_to_test:
    # Create a contingency table
    contingency_table = pd.crosstab(train_data[var], train_data['hypertension'])
    
    # Perform Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # Format the p-value
    formatted_p = "<0.001" if p < 0.001 else f"{p:.6f}"
    
    # Compute frequencies and percentages
    total_0 = contingency_table[0].sum()
    total_1 = contingency_table[1].sum()
    freq_0 = contingency_table[0]
    freq_1 = contingency_table[1]
    percent_0 = (freq_0 / total_0 * 100).round(2).astype(str) + "%"
    percent_1 = (freq_1 / total_1 * 100).round(2).astype(str) + "%"
    
    # Combine freq and percent in one string per cell
    freq_percent_0 = freq_0.astype(str) + " (" + percent_0 + ")"
    freq_percent_1 = freq_1.astype(str) + " (" + percent_1 + ")"
    
    # Create DataFrame for current variable
    temp_df = pd.DataFrame({
        'Variable': var,
        'Freq_0 (Percent)': freq_percent_0,
        'Freq_1 (Percent)': freq_percent_1,
        'p-value': [formatted_p] * len(freq_0)  # Repeat formatted p-value for each row
    }, index=freq_0.index)
    
    # Append current results to the main DataFrame
    results_df = pd.concat([results_df, temp_df])

# Output the table and reset index for clarity
print(results_df.reset_index(drop=True))

# Save to CSV
results_df.to_csv('/Users/xxx/Desktop/LISS data/patient_char.csv')

In [ ]:
# Define the list of variables to test
variables_to_test = [
    'age_group', 'male'
]

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

# Perform Chi-square test for each variable
for var in variables_to_test:
    # Create a contingency table
    contingency_table = pd.crosstab(test_data[var], test_data['hypertension'])
    
    # Perform Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # Format the p-value
    formatted_p = "<0.001" if p < 0.001 else f"{p:.6f}"
    
    # Compute frequencies and percentages
    total_0 = contingency_table[0].sum()
    total_1 = contingency_table[1].sum()
    freq_0 = contingency_table[0]
    freq_1 = contingency_table[1]
    percent_0 = (freq_0 / total_0 * 100).round(2).astype(str) + "%"
    percent_1 = (freq_1 / total_1 * 100).round(2).astype(str) + "%"
    
    # Combine frequency and percentage into a single string per cell
    freq_percent_0 = freq_0.astype(str) + " (" + percent_0 + ")"
    freq_percent_1 = freq_1.astype(str) + " (" + percent_1 + ")"
    
    # Create a DataFrame for the current variable
    temp_df = pd.DataFrame({
        'Variable': var,
        'Freq_0 (Percent)': freq_percent_0,
        'Freq_1 (Percent)': freq_percent_1,
        'p-value': [formatted_p] * len(freq_0)  # Repeat formatted p-value for each row
    }, index=freq_0.index)
    
    # Append the current results to the main DataFrame
    results_df = pd.concat([results_df, temp_df])

# Output the results and reset index for better readability
print(results_df.reset_index(drop=True))

In [ ]:
# List of variables to test
variables_to_test = [
    'edu_cls', 
    'depression_y', 
    'depression', 
    'anxiety', 
    'sleeping_prb', 
    'happiness', 
    'calmness', 
    'age_group', 
    'BMI',
    'life_statistician_x'
]

# Perform Kruskal-Wallis test for each variable
for var in variables_to_test:
    # Group data by 'hypertension' and extract values for each group
    groups = [group[var].values for name, group in train_data.groupby('hypertension')]
    
    # Conduct Kruskal-Wallis test
    stat, p = kruskal(*groups)
    
    # Output results with formatted decimal places
    print(f"\nVariable: {var}")
    print("H-statistic:", "{:.6f}".format(stat))
    print("p-value:", "{:.6f}".format(p))

### **Exploratory Data Analysis**

In [ ]:
# Histogram of age group

age_groups = ['< 45', '45 - 54', '55 - 64', '65 - 74', '>= 75']
non_hypertensive_counts = [155, 71, 87, 88, 56]
hypertensive_counts = [7, 15, 37, 67, 57]
total_counts = [sum(x) for x in zip(non_hypertensive_counts, hypertensive_counts)]

# Calculating percentages
non_hypertensive_percentages = [(count / total * 100) for count, total in zip(non_hypertensive_counts, total_counts)]
hypertensive_percentages = [(count / total * 100) for count, total in zip(hypertensive_counts, total_counts)]

# Setting up the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting the non-hypertensive bars
bar1 = ax.bar(age_groups, non_hypertensive_percentages, label='Non-Hypertensive', color='blue', alpha=0.7)

# Plotting the hypertensive bars on top of non-hypertensive bars
bar2 = ax.bar(age_groups, hypertensive_percentages, bottom=non_hypertensive_percentages, label='Hypertensive', color='red', alpha=0.7)

# Adding data labels to the bars
for bar in bar1:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval / 2, f'{non_hypertensive_counts[bar1.index(bar)]} ({yval:.2f}%)', ha='center', va='center', color='white', fontweight='bold')

for bar in bar2:
    yval = bar.get_height() + bar1[bar2.index(bar)].get_height()
    bottom = bar1[bar2.index(bar)].get_height()
    ax.text(bar.get_x() + bar.get_width()/2, bottom + bar.get_height()/2, f'{hypertensive_counts[bar2.index(bar)]} ({yval - bottom:.2f}%)', ha='center', va='center', color='white', fontweight='bold')

# Adding labels and title
ax.set_ylabel('Percentage')
ax.set_xlabel('Age Group')
# ax.set_title('Distribution of Hypertensive and Non-Hypertensive Individuals by Age Group')
ax.legend()


# outfile path
plt.savefig('/Users/xxx/Desktop/LISS data/results/figure_1.png')

# Display the plot
plt.show()


In [ ]:
# Histogram of sex

Sex = ['Female', 'Male']
non_hypertensive_counts = [247, 210]
hypertensive_counts = [82, 101]
total_counts = [sum(x) for x in zip(non_hypertensive_counts, hypertensive_counts)]

# Calculating percentages
non_hypertensive_percentages = [(count / total * 100) for count, total in zip(non_hypertensive_counts, total_counts)]
hypertensive_percentages = [(count / total * 100) for count, total in zip(hypertensive_counts, total_counts)]

# Setting up the figure and axes
fig, ax = plt.subplots(figsize=(8, 4))  # Smaller figure size

# Plotting the non-hypertensive bars
bar1 = ax.bar(Sex, non_hypertensive_percentages, width=0.2, label='Non-Hypertensive', color='blue', alpha=0.7)

# Plotting the hypertensive bars on top of non-hypertensive bars
bar2 = ax.bar(Sex, hypertensive_percentages, width=0.2, bottom=non_hypertensive_percentages, label='Hypertensive', color='red', alpha=0.7)

# Adding data labels to the bars
for bar in bar1:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval / 2, f'{non_hypertensive_counts[bar1.index(bar)]} ({yval:.2f}%)', ha='center', va='center', color='white', fontweight='bold', fontsize=8)

for bar in bar2:
    yval = bar.get_height() + bar1[bar2.index(bar)].get_height()
    bottom = bar1[bar2.index(bar)].get_height()
    ax.text(bar.get_x() + bar.get_width()/2, bottom + bar.get_height()/2, f'{hypertensive_counts[bar2.index(bar)]} ({yval - bottom:.2f}%)', ha='center', va='center', color='white', fontweight='bold', fontsize=8)

# Adjusting label sizes
ax.set_ylabel('Percentage', fontsize=10)
ax.set_xlabel('Sex', fontsize=10)
# ax.set_title('Distribution of Hypertensive and Non-Hypertensive Individuals by Sex', fontsize=12)
ax.legend(fontsize=10)
ax.tick_params(axis='both', which='major', labelsize=8)  # Adjusting tick labels size

# outfile path
plt.savefig('/Users/xxx/Desktop/LISS data/results/figure_2.png')

# Display the plot
plt.show()


In [ ]:
# Confusion matrix (number)

# Data for multiple models with corrected confusion matrix structure
data = [
    ('Baseline', np.array([[71, 112], [37, 420]])),  # TP, FN, FP, TN
    ('Model 2', np.array([[140, 43], [145, 312]])),  # TP, FN, FP, TN
    ('Model 3', np.array([[132, 51], [121, 336]])),  # TP, FN, FP, TN
    ('Model 4', np.array([[151, 32], [155, 302]]))   # TP, FN, FP, TN
]

# Set up the plot configuration for 2 rows and 2 columns
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))  # Two rows, two columns with a specified figsize

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Plot each model's confusion matrix
for idx, (model_label, matrix) in enumerate(data):
    ax = axes[idx]

    # Display the matrix using a blue color map
    cax = ax.matshow(matrix, cmap=plt.cm.Blues, aspect=0.4)  # Adjust aspect ratio to make matrices more rectangular

    # Set tick marks and labels with increased font size
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Positive', 'Negative'], fontsize=12)  # Adjusted font size for readability
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['Positive', 'Negative'], fontsize=12)  # Adjusted font size for readability
    ax.set_xlabel('Predicted', fontsize=12)  # Adjusted font size
    ax.set_ylabel('Actual', fontsize=12)  # Adjusted font size
    ax.set_title(model_label, fontsize=14)  # Adjusted font size

    # Annotate the matrix with absolute values, ensuring readability
    for (i, j), val in np.ndenumerate(matrix):
        text_color = 'white' if val > np.max(matrix)/2 else 'black'
        ax.text(j, i, f'{val}', ha='center', va='center', color=text_color, fontsize=14)  # Increased font size for visibility

plt.tight_layout()  # Optimize the layout to avoid overlap and ensure clarity
plt.savefig('/Users/xxx/Desktop/LISS data/results/f_cf_number.png')
plt.show()


In [ ]:
# Confusion matrix (percentage)
'
# Data for multiple models with corrected confusion matrix structure
data = [
    ('Baseline', np.array([[71, 112], [37, 420]])),  # TP, FN, FP, TN
    ('Model 2', np.array([[140, 43], [145, 312]])),  # TP, FN, FP, TN
    ('Model 3', np.array([[132, 51], [121, 336]])),  # TP, FN, FP, TN
    ('Model 4', np.array([[151, 32], [155, 302]]))   # TP, FN, FP, TN
]

# Set up the plot configuration for 2 rows and 2 columns
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))  # Two rows, two columns with a specified figsize

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Plot each model's confusion matrix
for idx, (model_label, matrix) in enumerate(data):
    ax = axes[idx]
    # Calculate percentages for each cell based on row totals
    row_sums = matrix.sum(axis=1, keepdims=True)
    percentage_matrix = 100 * matrix.astype('float') / row_sums

    # Display the matrix using a blue color map
    cax = ax.matshow(percentage_matrix, cmap=plt.cm.Blues, aspect=0.4)  # Adjust aspect ratio to make matrices more rectangular

    # Set tick marks and labels with increased font size
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Positive', 'Negative'], fontsize=12)  # Adjusted font size for readability
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['Positive', 'Negative'], fontsize=12)  # Adjusted font size for readability
    ax.set_xlabel('Predicted', fontsize=12)  # Adjusted font size
    ax.set_ylabel('Actual', fontsize=12)  # Adjusted font size
    ax.set_title(model_label, fontsize=14)  # Adjusted font size

    # Annotate the matrix with percentage values, ensuring readability
    for (i, j), val in np.ndenumerate(percentage_matrix):
        text_color = 'white' if val > 50 else 'black'
        ax.text(j, i, f'{val:.1f}%', ha='center', va='center', color=text_color, fontsize=14)  # Increased font size for visibility

plt.tight_layout()  # Optimize the layout to avoid overlap and ensure clarity
plt.savefig('/Users/xxx/Desktop/LISS data/results/f_cf_rate.png')
plt.show()


In [ ]:
# Confusion matrix stratified by sex

# Data for multiple models with corrected confusion matrix structure
data = [
    ('Baseline Male', np.array([[44, 57], [18, 192]])),  
    ('Baseline Female', np.array([[27, 55], [19, 228]])), 
    ('Model 2 Male', np.array([[83, 18], [78, 132]])),    
    ('Model 2 Female', np.array([[57, 25], [67, 180]])), 
    ('Model 3 Male', np.array([[76, 25], [67, 143]])),   
    ('Model 3 Female', np.array([[56, 26], [54, 193]])),  
    ('Model 4 Male', np.array([[79, 22], [75, 135]])),   
    ('Model 4 Female', np.array([[58, 24], [54, 193]]))   # Corrected order: TP, FN, FP, TN
]

# Set up the plot configuration for 2 rows and 4 columns
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 6))  # Adjusted figsize to make plots more rectangular and compact

for i, (label, matrix) in enumerate(data):
    row = i // 4  # Determine row: 0 for males, 1 for females
    col = i % 4   # Column index loops every four
    ax = axes[row, col]
    cax = ax.matshow(matrix, cmap=plt.cm.Blues, aspect=0.4)  # Adjust aspect to make the matrix more rectangular

    # Set tick marks and labels with increased font size
    classes = ['Positive', 'Negative']
    ax.set_xticks([0, 1])
    ax.set_xticklabels(classes, fontsize=12)
    ax.set_yticks([0, 1])
    ax.set_yticklabels(classes, fontsize=12)
    ax.set_xlabel('Predicted', fontsize=12)
    ax.set_ylabel('Actual', fontsize=12)
    ax.set_title(label, fontsize=14)

    # Annotate the matrix with absolute values
    for (i, j), val in np.ndenumerate(matrix):
        text_color = 'white' if val > np.max(matrix)/2 else 'black'
        ax.text(j, i, f'{val}', ha='center', va='center', color=text_color, fontsize=14)  # Adjusted font size for visibility

plt.tight_layout()  # Optimize the layout to avoid overlap and ensure clarity
plt.savefig('/Users/xxx/Desktop/LISS data/results/f_cf_sex_number.png')
plt.show()


In [ ]:
# Confusion matrix stratified by age group

# Data for multiple age groups across different models
models = {
    'Baseline': {
        '< 45': [0, 7, 0, 155],  # [TP, FN, FP, TN]
        '45 - 54': [0, 15, 0, 71],
        '55 - 64': [7, 30, 3, 84],
        '65 - 74': [26, 41, 14, 75],
        '>= 75': [38, 19, 21, 35]
    },
    'Model 2': {
        '< 45': [0, 7, 2, 153],
        '45 - 54': [3, 12, 4, 67],
        '55 - 64': [31, 6, 31, 56],
        '65 - 74': [53, 14, 66, 22],
        '>= 75': [53, 4, 42, 14]
    },
    'Model 3': {
        '< 45': [0, 7, 1, 154],
        '45 - 54': [3, 12, 3, 68],
        '55 - 64': [23, 14, 19, 68],
        '65 - 74': [52, 15, 61, 27],
        '>= 75': [54, 3, 37, 19]
    },
    'Model 4': {
        '< 45': [1, 6, 1, 154],
        '45 - 54': [5, 10, 5, 66],
        '55 - 64': [22, 15, 20, 67],
        '65 - 74': [56, 11, 58, 30],
        '>= 75': [53, 4, 45, 11]
    }
}

# Calculate total number of plots needed
total_plots = sum(len(v) for v in models.values())
n_cols = 4  # Number of columns per row
n_rows = (total_plots + n_cols - 1) // n_cols  # Calculate rows needed

# Set up the plot configuration
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 14), squeeze=False)

# Flatten the axes for easier management
axes = axes.flatten()

# Plotting loop
plot_idx = 0
for model_name, age_groups in models.items():
    for group_name, values in age_groups.items():
        ax = axes[plot_idx]
        matrix = np.array(values).reshape((2, 2))
        cax = ax.matshow(matrix, cmap=plt.cm.Blues, aspect=0.4)  # Use a bluer color map with adjusted aspect

        # Set ticks and labels
        ax.set_xticks([0, 1])
        ax.set_xticklabels(['Positive', 'Negative'], fontsize=12)
        ax.set_yticks([0, 1])
        ax.set_yticklabels(['Positive', 'Negative'], fontsize=12)
        ax.set_xlabel('Predicted', fontsize=12)
        ax.set_ylabel('Actual', fontsize=12)
        ax.set_title(f'{model_name} {group_name}', fontsize=14)

        # Annotate the matrix with absolute values
        for (i, j), val in np.ndenumerate(matrix):
            text_color = 'white' if val > max(matrix.flatten()) / 2 else 'black'
            ax.text(j, i, f'{val}', ha='center', va='center', color=text_color, fontsize=14)

        plot_idx += 1


# Adjust subplots
plt.tight_layout(pad=0.4, w_pad=0.1, h_pad=1.0)
fig.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.1, hspace=0.4)

# Hide any unused axes
for i in range(plot_idx, n_rows * n_cols):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.savefig('/Users/xxx/Desktop/LISS data/results/f_cf_age_number.png')
plt.show()


In [ ]:
# Line chart of F1-score, Precision, and Recall across different age groups

# Prepare data
data = {
    'Model': ['Baseline']*5 + ['Model 2']*5 + ['Model 3']*5 + ['Model 4']*5,
    'Age Group': ['<45', '45 - 54', '55 - 64', '65 - 74', '>= 75'] * 4,
    'F1': [0, 0, 0.2979, 0.4906, 0.6552, 0, 0.2727, 0.6263, 0.5699, 0.6974,
           0, 0.2857, 0.5823, 0.5778, 0.7297, 0.2222, 0.4, 0.557, 0.6188, 0.6839],
    'Recall': [0, 0, 0.1892, 0.3881, 0.6667, 0, 0.2, 0.8378, 0.791, 0.9298,
               0, 0.2, 0.6216, 0.7761, 0.9474, 0.1429, 0.3333, 0.5946, 0.8358, 0.9298],
    'Precision': [0, 0, 0.7, 0.6667, 0.6441, 0, 0.4286, 0.5, 0.4454, 0.5579,
                  0, 0.5, 0.5476, 0.4602, 0.5934, 0.5, 0.5, 0.5238, 0.4912, 0.5408]
}

# Create DataFrame
df = pd.DataFrame(data)

# Set Age Group as an ordered categorical variable
age_categories = ['<45', '45 - 54', '55 - 64', '65 - 74', '>= 75']
df['Age Group'] = pd.Categorical(df['Age Group'], categories=age_categories, ordered=True)

# Transform data from wide format to long format for plotting
df_melted = df.melt(id_vars=['Model', 'Age Group'], var_name='Metric', value_name='Value')

# Set Metric as an ordered categorical variable
metric_categories = ['F1', 'Precision', 'Recall']
df_melted['Metric'] = pd.Categorical(df_melted['Metric'], categories=metric_categories, ordered=True)

# Set up the plot layout: two rows and two columns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Plot performance metrics for each model
models = df_melted['Model'].unique()
for ax, model in zip(axes, models):
    model_data = df_melted[df_melted['Model'] == model]
    pivot_df = model_data.pivot(index='Age Group', columns='Metric', values='Value')
    pivot_df.plot(kind='line', marker='o', ax=ax, title=f'Performance Metrics for {model}')  # Line plot with markers
    ax.set_xlabel('Age Group')
    ax.set_ylabel('Values')
    ax.legend(title='Metrics')

    # Add numerical labels on each data point
    for line in ax.get_lines():
        for x, y in zip(line.get_xdata(), line.get_ydata()):
            ax.annotate(f'{y:.3f}', xy=(x, y), textcoords="offset points", xytext=(0, 5), ha='center')

    plt.xticks(range(len(age_categories)), labels=age_categories, rotation=45)

plt.tight_layout()
plt.savefig('/Users/xxx/Desktop/LISS data/results/f_age_performance.png')
plt.show()

In [ ]:
# Prepare data
data = {
    'Model': ['Baseline', 'Baseline', 'Model 2', 'Model 2', 'Model 3', 'Model 3', 'Model 4', 'Model 4'],
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female'],
    'F1': [0.540, 0.422, 0.634, 0.553, 0.623, 0.583, 0.620, 0.598],
    'Recall': [0.436, 0.329, 0.822, 0.695, 0.753, 0.683, 0.782, 0.707],
    'Precision': [0.710, 0.587, 0.516, 0.460, 0.532, 0.509, 0.513, 0.518]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert data from wide format to long format for plotting
df_melted = df.melt(id_vars=['Model', 'Gender'], var_name='Metric', value_name='Value')

# Set up the figure layout, creating one plot per model
fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Define bar width
width = 0.4

# Plot each model's metrics
for ax, (model, model_group) in zip(axes, df_melted.groupby('Model')):
    metrics = model_group['Metric'].unique()
    x = range(len(metrics))  # Base positions on x-axis

    # Male data
    male_data = model_group[model_group['Gender'] == 'Male']
    ax.bar([xi - width/2 for xi in x], male_data['Value'], width=width, label='Male', color='blue', alpha=0.7)

    # Female data
    female_data = model_group[model_group['Gender'] == 'Female']
    ax.bar([xi + width/2 for xi in x], female_data['Value'], width=width, label='Female', color='red', alpha=0.7)

    # Add data labels
    for xi, value in zip([xi - width/2 for xi in x], male_data['Value']):
        ax.text(xi, value, f'{value:.2f}', ha='center', va='bottom', fontsize=14)
    for xi, value in zip([xi + width/2 for xi in x], female_data['Value']):
        ax.text(xi, value, f'{value:.2f}', ha='center', va='bottom', fontsize=14)

    ax.set_title(f'Performance Metrics for {model}')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()

plt.tight_layout()
plt.show()
